In [108]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'C:\\Users\\whcao'

In [109]:
data_train = pd.read_csv('train.csv',parse_dates=['purchase_date','release_date'])
data_test = pd.read_csv('test.csv',parse_dates=['purchase_date','release_date'])

In [110]:
# Check the lack of data
def null_table(df):
        null_val = df.isnull().sum()
        percent = 100 * df.isnull().sum()/len(df)
        kesson_table = pd.concat([null_val, percent], axis =1)
        kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : 'lack', 1 :'%'})
        return kesson_table_ren_columns
    
null_table(data_train)

,lack,%
id,0,0.000000
playtime_forever,0,0.000000
is_free,0,0.000000
price,0,0.000000
genres,0,0.000000
categories,0,0.000000
tags,0,0.000000
purchase_date,2,0.560224
release_date,0,0.000000
total_positive_reviews,2,0.560224


In [111]:
null_table(data_test)

,lack,%
id,0,0.000000
is_free,0,0.000000
price,0,0.000000
genres,0,0.000000
categories,0,0.000000
tags,0,0.000000
purchase_date,2,2.222222
release_date,0,0.000000
total_positive_reviews,2,2.222222
total_negative_reviews,2,2.222222


In [112]:
# fill the lack total_positive_reviews and total_negative_reviews with mean value
meanvalue1 = data_train['total_positive_reviews'].mean()
data_train['total_positive_reviews'].fillna(meanvalue1, inplace=True)
meanvalue2 = data_test['total_positive_reviews'].mean()
data_test['total_positive_reviews'].fillna(meanvalue2, inplace=True)
meanvalue3 = data_train['total_negative_reviews'].mean()
data_train['total_negative_reviews'].fillna(meanvalue3, inplace=True)
meanvalue4 = data_test['total_negative_reviews'].mean()
data_test['total_negative_reviews'].fillna(meanvalue4, inplace=True)

In [113]:
# fill the lack purchase_date with release_date
lack1= data_train[data_train['purchase_date'].isnull()]
lack2= data_test[data_test['purchase_date'].isnull()]
ids1 = list(lack1['id'])
ids2 = list(lack2['id'])
for id in ids1:
    data_train.loc[id,'purchase_date'] = data_train.loc[id,'release_date']
for id in ids2:
    data_test.loc[id,'purchase_date'] = data_test.loc[id,'release_date']

In [114]:
# use 0/1 to encode is_free features
mapping = {False: 1,True: 0}
data_train["is_free"] = data_train["is_free"].map(mapping)
data_test["is_free"] = data_test["is_free"].map(mapping)

In [115]:
# one-hot encoding for category features "genres","categories" and "tags" 
data_combine = pd.concat([data_train.drop(['playtime_forever'], axis=1), data_test],axis=0,ignore_index=True)
genre = data_combine['genres'].str.get_dummies(',')
category = data_combine['categories'].str.get_dummies(',')
tag = data_combine['tags'].str.get_dummies(',')

In [116]:
num_genre = genre.shape[1]    # 20 
num_category = category.shape[1] # 29
num_tag = tag.shape[1]  # 317

In [117]:
# For there are too many items, I choose to drop those infrequent ones
for item in genre.columns:
    if pd.value_counts(genre[item])[1]<200:
        genre.drop(item, axis=1, inplace=True)
for item in category.columns:
    if pd.value_counts(category[item])[1]<200:
        category.drop(item, axis=1, inplace=True)
for item in tag.columns:
    if pd.value_counts(tag[item])[1]<200:
        tag.drop(item, axis=1, inplace=True)

In [118]:
data_combine = pd.concat([data_combine, genre, category, tag], axis=1)
data_combine.drop(['genres','categories','tags'], axis=1, inplace=True)

In [119]:
data_combine['purchase_year'] = data_combine['purchase_date'].apply(lambda x: x.year)
data_combine['release_year'] = data_combine['release_date'].apply(lambda x:x.year)
#data_combine.drop(['purchase_date', 'release_date'], axis=1,inplace=True)

In [120]:
data_combine["gap"] = data_combine["purchase_year"]- data_combine["release_year"]   # evaluate the time between the game released and the player purchased it
data_combine["freshness"] = 2019 - data_combine["release_year"]   # evaluate how new the game is
data_combine["holding"] = 2019 - data_combine["purchase_year"]  #evaluate how long the player has held the game
data_combine.drop(['purchase_year', 'release_year'], axis=1,inplace=True)

In [121]:
# Min-Max Normalization
def min_max_normalization(df,columns):
    for column in columns:
        df[column] = (df[column]-df[column].min())/(df[column].max()-df[column].min())
        
min_max_normalization(data_combine,['price','total_positive_reviews','total_negative_reviews','gap','freshness','holding','purchase_date', 'release_date'])

In [122]:
data_combine = data_combine.loc[:,~data_combine.columns.duplicated()]
train_num = data_train.shape[0]
test_num = data_test.shape[0]
train_x = data_combine.iloc[:train_num].drop(['id'], axis=1)
train_y = data_train['playtime_forever']
test_x = data_combine.iloc[train_num:].drop(['id'], axis=1)

In [123]:
import xgboost as xgb
from pandas import DataFrame
bst = xgb.XGBRegressor(seed = 1850)
bst.fit(train_x, train_y)
test_y = bst.predict(test_x)

[22:28:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


D:\anaconda\anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\anaconda\anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [124]:
# use the date from training set to predict and score
from sklearn import metrics
train_time = bst.predict(train_x)  
metrics.r2_score(train_y.values, train_time)

0.9521273564577646

In [125]:
from xgboost import XGBRegressor
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [126]:
def modelfit(alg, train_x, train_y, useTrainCV = True, cv_folds = 5, early_stopping_rounds =  50):
    alg.fit(train_x, train_y,eval_metric = 'rmse')
    train_predictions = alg.predict(train_x)  # predict the training set
    print ("\nAbout this model")
    print ("r2_score : %.4g" % metrics.r2_score(train_y.values, train_predictions))

In [127]:
xgb1 = XGBRegressor(base_score = 0.5,
                   colsample_bylevel = 1,
                   colsample_bytree = 1,
                   gamma = 0,
                   learning_rate = 0.01,
                   max_delta_step = 0,
                   max_depth = 10,
                   min_child_weight = 2,
                   missing = None,
                   n_estimators = 5000,
                   nthread = -1,
                   objective = 'reg:linear',
                   reg_alpha = 0,
                   reg_lambda = 1,
                   scale_pos_weight = 1,
                   seed = 1850,
                   slient = True,
                   sumsample = 1)
modelfit(xgb1, train_x, train_y)

D:\anaconda\anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\anaconda\anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:28:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

About this model
r2_score : 1


In [128]:
test_y = xgb1.predict(test_x)
test_y[test_y<0]=0

In [129]:
#from sklearn.ensemble import RandomForestRegressor
#random_forest = RandomForestRegressor(n_estimators=8000)
#random_forest.fit(train_x, train_y)
#test_y1 = random_forest.predict(test_x)
#test_y1[test_y<0]=0

from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=500)
gbm.fit(train_x, train_y)
test_y1 = gbm.predict(test_x)
test_y1[test_y1<0]=0

In [130]:
ids = [str(n) for n in range(test_num)]
ids = np.asarray(ids)
predict_outcome = np.vstack((ids,(test_y+test_y1)/2)).T
submission = pd.DataFrame(data = predict_outcome,columns = ['id','playtime_forever'])
submission.to_csv('submission.csv', index = False)